**Get Data**

**ALL FUNCTIONS**

In [0]:
custom_spec_list = ['Specifications','Processor Type','Hard Drive Capacity','Condition','Product Dimensions (L x W x H)','Processor Speed','Manufacturer Part Number','Wireless Technology','RAM Memory','Maximum RAM Supported'
,'Operating System','Model','Screen Size','Brand',' Inches','Supported Media Format','See Description']

def preprocess(pdf):
  custom_spec_list = ['Specifications','Processor Type','Hard Drive Capacity','Condition','Product Dimensions (L x W x H)','Processor Speed','Manufacturer Part Number','Wireless Technology','RAM Memory','Maximum RAM Supported'
  ,'Operating System','Model','Screen Size','Brand',' Inches','Supported Media Format','See Description']
  for i in custom_spec_list:
    pdf=str(pdf).lstrip().rstrip().replace(i,' ')
  return pdf

def corpus_create(corpus,p):
  for i in p:
    corpus.append(i)
  
    

**Mount drive**

In [0]:
#Mount google drive to upload file
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from google.colab import drive
drive.mount('/content/drive')

**All Imports**

In [0]:
import pandas as pd
!pip install stop_words
from stop_words import get_stop_words
import re
from sklearn.feature_extraction.text import TfidfVectorizer

**Read in the scraped data**

In [0]:
df = pd.read_excel('/content/all_data.xlsx')
df.head()
df['Category'] = 0

**Category Path - y**


**Very Imp - Removing products which don't have a path**

In [0]:
df = df[df['product_path'].notnull()]  

In [0]:
df['Category'] = df['product_path'].apply(lambda x: str(x).split('/')[1])

**Dataset Final - Product Name, Category, ATI, Specs**

**Presently working with specs** 

In [0]:

final = df[['Category','product_name','product_ati','product_specs']]
final.head()

,Category,product_name,product_ati,product_specs
0,Computers,Refurbished Lenovo THINKCENTRE M92P-SFF Deskto...,Do as much as you like with this Lenovo THINKC...,SpecificationsBrandLenovoOperating SystemWindo...
1,Computers,Refurbished Dell Optiplex 790-SFF WA1-0309 Des...,Certified Refurbished Items: \n \n What to Ex...,SpecificationsBrandDellOperating SystemWindows...
2,Computers,HP 8200 Elite Desktop Computer with Windows 10...,This HP desktop computer is very fast and depe...,SpecificationsBrandHPOperating SystemWindows 1...
3,Computers,"Apple iMac 21.5"" 2.7GHz Core i5 (ME086LL/A) Al...","The iMac ""Core i5"" 2.7 21.5-Inch Aluminum (Lat...",SpecificationsBrandAppleOperating SystemMac OS...
4,Computers,Refurbished - Dell Optiplex Desktop Computer 3...,Refurbished - Dell Optiplex Desktop Computer 3...,SpecificationsBrandDellOperating SystemWindows...


**Preprocessed the SPECS with a user defined functions**

In [0]:
final['cleaned_specs']= final['product_specs']
final['cleaned_specs'] = final['cleaned_specs'].apply(preprocess)
final.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Category,product_name,product_ati,product_specs,cleaned_specs
0,Computers,Refurbished Lenovo THINKCENTRE M92P-SFF Deskto...,Do as much as you like with this Lenovo THINKC...,SpecificationsBrandLenovoOperating SystemWindo...,Lenovo Windows 10 4 GB M92P Intel Core i5-3470...
1,Computers,Refurbished Dell Optiplex 790-SFF WA1-0309 Des...,Certified Refurbished Items: \n \n What to Ex...,SpecificationsBrandDellOperating SystemWindows...,Dell Windows 10 8 GB 790 Intel Core i5-2500 Pr...
2,Computers,HP 8200 Elite Desktop Computer with Windows 10...,This HP desktop computer is very fast and depe...,SpecificationsBrandHPOperating SystemWindows 1...,HP Windows 10 4.00 GB 8200 Elite Intel Quad Co...
3,Computers,"Apple iMac 21.5"" 2.7GHz Core i5 (ME086LL/A) Al...","The iMac ""Core i5"" 2.7 21.5-Inch Aluminum (Lat...",SpecificationsBrandAppleOperating SystemMac OS...,"Apple Mac OS 10.12 Sierra, Mac OS 8 GB ME086LL..."
4,Computers,Refurbished - Dell Optiplex Desktop Computer 3...,Refurbished - Dell Optiplex Desktop Computer 3...,SpecificationsBrandDellOperating SystemWindows...,Dell Windows 10 4.00 GB K_00180 Intel Core 2 D...


**Baseline Model - tf-idf**

**Creating corpus and using td-idf vectorizer**

In [0]:
#count vectorizer 
corpus = []
corpus_create(corpus,final['cleaned_specs'])
vectorizer = TfidfVectorizer()
tfidf_final = vectorizer.fit_transform(corpus)

In [0]:
corp = pd.DataFrame()
corp['collected'] = list(tfidf_final.toarray())
fin_l=[]
for i in corp['collected']:
  fin_l.append(list(i))

df1 = pd.DataFrame(fin_l)


In [0]:
df1.shape
#df1.to_csv('tf-idfValues')

(13085, 20243)

**Adding feature names to tf-idf matrix**

In [0]:
a = vectorizer.vocabulary_
len(a.keys())
df1.columns = a.keys()
df1['product_name'] = final['product_name']

In [0]:
df1['product_name'] = final['product_name']
df1

**Final Dataset - df1**

In [0]:
#df1.to_csv('Final_tf-idf_df')
import pandas as pd
df1 = pd.read_csv('/content/Final_tf-idf_df')

In [0]:
df1= df1.loc[:, df1.columns != 'Unnamed: 0']
df1

**Td-idf Vectorization done**

**Second Part - PCA, LDA, t-SNE, AE**


**PCA**

**y variable**

In [0]:
final['Category'].unique()

In [0]:
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA  
sc = StandardScaler() 

def PCArun(df,testdata,x):
  # Creating train and test set
  X_train, X_test, y_train, y_test = train_test_split(df, testdata, test_size = 0.2, random_state = 0) 

  #Fitting model  
  X_train = sc.fit_transform(X_train) 
  X_test = sc.transform(X_test) 

  # Applying PCA function on training for 80% variance 
  pca = PCA(x) 
  
  X_train = pca.fit_transform(X_train) 
  X_test = pca.transform(X_test)  
  explained_variance = pca.explained_variance_ratio_ 
  return X_train, X_test,y_train,y_test,explained_variance




**Random Forest for final output**

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score, f1_score

**1. RF for Non PCA Data**

In [0]:
X_train_non_pca, X_test_non_pca,y_train_non_pca, y_test_non_pca, = train_test_split(df1.loc[:, df1.columns != 'product_name'],final['Category'], test_size = 0.2, random_state = 0) 


classifier_non_pca = RandomForestClassifier(max_depth=2, random_state=0,)
classifier_non_pca.fit(X_train_non_pca, y_train_non_pca)

# Predicting the Test set results
y_pred_non_pca = classifier_non_pca.predict(X_test_non_pca)

cm_non_pca = confusion_matrix(y_test_non_pca, y_pred_non_pca)

print(f1_score(y_test_non_pca, y_pred_non_pca,average='weighted'))
print(accuracy_score(y_test_non_pca, y_pred_non_pca))


0.43415696568428047
0.5869316010699274


**2. RF for PCA Data - 0.7**

In [0]:

X_train_seven, X_test_seven,y_train_seven,y_test_seven, explained_variance_seven = PCArun(df1.loc[:, df1.columns != 'product_name'],final['Category'],0.7)
classifier_seven = RandomForestClassifier(max_depth=2, random_state=0,)
classifier_seven.fit(X_train_seven, y_train_seven)
# Predicting the Test set results
y_pred_seven = classifier.predict(X_test_seven)
#y_pred
cm = confusion_matrix(y_test_seven, y_pred_seven)

print("f1 Score:", round(f1_score(y_test_seven, y_pred_seven,average='weighted'),4), "\n"
     "Accuracy:", round(accuracy_score(y_test_seven, y_pred_seven),4))

f1 Score: 0.7002 
Accuracy: 0.7711


**3. RF for PCA Data - 0.8**

**PCA gives us 3097 dimensions for 80% variance**


In [0]:
X_train_eight, X_test_eight,y_train_eight,y_test_eight, explained_variance_eight = PCArun(df1.loc[:, df1.columns != 'product_name'],final['Category'],0.8)
classifier_eight = RandomForestClassifier(max_depth=2, random_state=0,)
classifier_eight.fit(X_train_eight, y_train_eight)
# Predicting the Test set results
y_pred_eight = classifier_eight.predict(X_test_eight)
#y_pred
cm = confusion_matrix(y_test_eight, y_pred_eight)

print("f1 Score:", round(f1_score(y_test_eight, y_pred_eight,average='weighted'),4), "\n"
     "Accuracy:", round(accuracy_score(y_test_eight, y_pred_eight),4))

f1 Score: 0.7013 
Accuracy: 0.7723


**XGBoost for final output**

In [0]:
from xgboost import XGBClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score, f1_score

**1. XGBOOST FOR Non PCA DATA**

In [0]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score, f1_score

In [0]:
from xgboost import XGBClassifier
xgb_non_pca = XGBClassifier()
xgb_non_pca.fit(X_train_non_pca, y_train_non_pca)
y_pred_test_non_pca = xgb_non_pca.predict(X_test_non_pca)
print(f1_score(y_test_non_pca ,y_pred_test_non_pca,average='weighted' ))
print(accuracy_score(y_test_non_pca ,y_pred_test_non_pca ))



**2. XGBOOST FOR PCA = 0.7**

In [0]:
xgb_seven = XGBClassifier()
xgb_seven .fit(X_train, y_train)
y_pred_train_seven  = xgb.predict(X_train_seven)
y_pred_test_seven  = xgb.predict(X_test_seven)
#y_pred_test_proba = xgb.predict_proba(x_test)[:,1]
print("training accuracy:", round(accuracy_score(y_train_seven ,y_pred_train_seven ),4), "\n"
     "test accuracy:", round(accuracy_score(y_test_seven ,y_pred_test_seven ),4))

**3. XGBOOST FOR PCA = 0.8**

In [0]:
xgb_eight = XGBClassifier()
xgb_eight.fit(X_train_eight, y_train_eight)
y_pred_train_eight = xgb.predict(X_train_eight)
y_pred_test_eight = xgb.predict(X_test_eight)
#y_pred_test_proba = xgb.predict_proba(x_test)[:,1]
print("training accuracy:", round(accuracy_score(y_train_eight,y_pred_train_eight),4), "\n"
     "test accuracy:", round(accuracy_score(y_test_eight,y_pred_test_eight),4))

**MLP for final output**

In [0]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score, f1_score

**1. MLP FOR Non PCA DATA**

In [0]:
mlp_non_pca = MLPClassifier()
mlp_non_pca .fit(X_train_non_pca , y_train_non_pca)
y_pred_test_non_pca  = mlp_non_pca.predict(X_test_non_pca)
print(f1_score(y_test_non_pca ,y_pred_test_non_pca,average='weighted' ))
print(accuracy_score(y_test_non_pca ,y_pred_test_non_pca ))

training accuracy: 0.9691 
test accuracy: 0.88


**2. MLP FOR PCA = 0.7**

In [0]:
mlp_seven = MLPClassifier()
mlp_seven.fit(X_train_seven, y_train_seven)
y_pred_test_seven = mlp_seven.predict(X_test_seven)
print(f1_score(y_test_seven ,y_pred_test_seven,average='weighted' ))
print(accuracy_score(y_test_seven ,y_pred_test_seven))

0.8582228702317342
0.8685517768437142


**3. MLP FOR PCA = 0.8**

In [0]:
mlp_eight = MLPClassifier()
mlp_eight.fit(X_train_eight, y_train_eight)
y_pred_test_eight = mlp_eight.predict(X_test_eight)
print(f1_score(y_test_eight ,y_pred_test_eight,average='weighted' ))
print(accuracy_score(y_test_eight ,y_pred_test_eight))

0.8597146762569609
0.8670233091325946


In [0]:
print(f1_score(y_test_eight ,y_pred_test_eight,average='weighted' ))

**Auto-encoding**

In [0]:
import seaborn as sns
from sklearn.metrics import log_loss
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from numpy.random import seed

from keras.layers import Input, Dense
from keras.models import Model

In [0]:
from sklearn.model_selection import train_test_split
y = final['pre_define_cat']
x_train,x_test,y_train,y_test=train_test_split(all_data_fastext,y,test_size=0.20,random_state=20)

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)

x_train_sc = scaler.transform(x_train)
x_test_sc = scaler.transform(x_test)

In [0]:
x_sc = scaler.transform(all_data_fastext)

In [0]:
def ae_setting(start_feature_num, end_feature_num,X_train,X_test,all_data_scaled):
  encoding_dim = end_feature_num
  ncol = start_feature_num
  middle_num = (start_feature_num+end_feature_num)//2

  input_dim = Input(shape = (ncol, ))

  # Encoder Layers
  encoded1 = Dense(middle_num, activation = 'relu')(input_dim)
  encoded2 = Dense(encoding_dim, activation = 'relu')(encoded1)

  # Decoder Layers
  decoded1 = Dense(middle_num, activation = 'relu')(encoded2)
  decoded2 = Dense(ncol, activation = 'sigmoid')(decoded1)

  # Combine Encoder and Deocder layers
  autoencoder = Model(inputs = input_dim, outputs = decoded2)

  # Compile the Model
  autoencoder.compile(optimizer = 'adadelta', loss = 'binary_crossentropy')

  print(autoencoder.summary())

  autoencoder.fit(X_train, X_train, nb_epoch = 30, batch_size = 32, shuffle = False, validation_data = (X_test, X_test))

  encoder = Model(inputs = input_dim, outputs = encoded2)
  encoded_input = Input(shape = (encoding_dim, ))

  encoded_x = pd.DataFrame(encoder.predict(all_data_scaled))
  encoded_x = encoded_x.add_prefix('feature_')

  return encoded_x

In [0]:
test_function = ae_setting(30,5,x_train_sc,x_test_sc,x_sc)